In [462]:
import pandas as pd  # For data manipulation and analysis
import numpy as np
from bs4 import BeautifulSoup  # For web scraping and HTML parsing
import requests  # To make HTTP requests to access webpage content
import matplotlib.pyplot as plt  # For plotting (not currently used in this code)

# Define the URL of the webpage to scrape mortgage rate data
url = 'https://www.ratehub.ca/best-mortgage-rates/5-year/variable'

# Send a GET request to fetch the HTML content of the webpage
page = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page.text, 'html')

# Find the table elements that contain the data we want, specifically looking for elements with class 'table-container'
table = soup.find_all('table', class_='table-container')

# Extract and clean the text from each table element
table = [i.text.strip() for i in table]

# Locate the table header and extract text from each column name (header cell)
header = soup.find('thead')
header_text = [th.get_text(strip=True) for th in header.find_all('th') if th.get_text(strip=True)]

# Create an empty DataFrame with the extracted column headers
df = pd.DataFrame(columns=header_text)

# Find all table rows within the <tbody> elements (contains actual rate data)
table_rows = soup.find_all('tbody')

# Initialize an empty list to store extracted row data
data = []

# Loop through each table row and extract text from the first three columns: Rate, Provider, and Payment
for row in table_rows:
    for tr in row.find_all('tr'):
        columns = tr.find_all('td')
        # Ensure there are enough columns before attempting to extract data
        if len(columns) >= 3:
            rate = columns[0].get_text(strip=True)
            provider = columns[1].get_text(strip=True)
            payment = columns[2].get_text(strip=True)
            # Append the extracted values as a new row in the data list
            data.append([rate, provider, payment])

# Create a DataFrame from the extracted data using the previously defined headers
df5v = pd.DataFrame(data, columns=header_text)
df5v.drop(columns=['Payment'], inplace=True)
df5v.rename(columns={'Rate': 'Variable rate 5y'}, inplace=True)

In [463]:
# Define the URL of the webpage to scrape mortgage rate data
url = 'https://www.ratehub.ca/best-mortgage-rates/3-year/variable'

# Send a GET request to fetch the HTML content of the webpage
page = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page.text, 'html')

# Find the table elements that contain the data we want, specifically looking for elements with class 'table-container'
table = soup.find_all('table', class_='table-container')

# Extract and clean the text from each table element
table = [i.text.strip() for i in table]

# Locate the table header and extract text from each column name (header cell)
header = soup.find('thead')
header_text = [th.get_text(strip=True) for th in header.find_all('th') if th.get_text(strip=True)]

# Create an empty DataFrame with the extracted column headers
df = pd.DataFrame(columns=header_text)

# Find all table rows within the <tbody> elements (contains actual rate data)
table_rows = soup.find_all('tbody')

# Initialize an empty list to store extracted row data
data = []

# Loop through each table row and extract text from the first three columns: Rate, Provider, and Payment
for row in table_rows:
    for tr in row.find_all('tr'):
        columns = tr.find_all('td')
        # Ensure there are enough columns before attempting to extract data
        if len(columns) >= 3:
            rate = columns[0].get_text(strip=True)
            provider = columns[1].get_text(strip=True)
            payment = columns[2].get_text(strip=True)
            # Append the extracted values as a new row in the data list
            data.append([rate, provider, payment])

# Create a DataFrame from the extracted data using the previously defined headers
df3v = pd.DataFrame(data, columns=header_text)
df3v.drop(columns=['Payment'], inplace=True)
df3v.rename(columns={'Rate': 'Variable rate 3y'}, inplace=True)

result_df=df5v.merge(df3v, on='Provider', how='outer')
result_df = result_df[['Provider', 'Variable rate 3y', 'Variable rate 5y']]

In [464]:
# Define the URL of the webpage to scrape mortgage rate data
url = 'https://www.ratehub.ca/best-mortgage-rates/1-year/fixed'

# Send a GET request to fetch the HTML content of the webpage
page = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page.text, 'html')

# Find the table elements that contain the data we want, specifically looking for elements with class 'table-container'
table = soup.find_all('table', class_='table-container')

# Extract and clean the text from each table element
table = [i.text.strip() for i in table]

# Locate the table header and extract text from each column name (header cell)
header = soup.find('thead')
header_text = [th.get_text(strip=True) for th in header.find_all('th') if th.get_text(strip=True)]

# Create an empty DataFrame with the extracted column headers
df = pd.DataFrame(columns=header_text)

# Find all table rows within the <tbody> elements (contains actual rate data)
table_rows = soup.find_all('tbody')

# Initialize an empty list to store extracted row data
data = []

# Loop through each table row and extract text from the first three columns: Rate, Provider, and Payment
for row in table_rows:
    for tr in row.find_all('tr'):
        columns = tr.find_all('td')
        # Ensure there are enough columns before attempting to extract data
        if len(columns) >= 3:
            rate = columns[0].get_text(strip=True)
            provider = columns[1].get_text(strip=True)
            payment = columns[2].get_text(strip=True)
            # Append the extracted values as a new row in the data list
            data.append([rate, provider, payment])

# Create a DataFrame from the extracted data using the previously defined headers
df1f = pd.DataFrame(data, columns=header_text)
df1f.drop(columns=['Payment'], inplace=True)
df1f.rename(columns={'Rate': 'Fixed rate 1y'}, inplace=True)

result_df=result_df.merge(df1f, on='Provider', how='outer')

In [465]:
# Define the URL of the webpage to scrape mortgage rate data
url = 'https://www.ratehub.ca/best-mortgage-rates/2-year/fixed'

# Send a GET request to fetch the HTML content of the webpage
page = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page.text, 'html')

# Find the table elements that contain the data we want, specifically looking for elements with class 'table-container'
table = soup.find_all('table', class_='table-container')

# Extract and clean the text from each table element
table = [i.text.strip() for i in table]

# Locate the table header and extract text from each column name (header cell)
header = soup.find('thead')
header_text = [th.get_text(strip=True) for th in header.find_all('th') if th.get_text(strip=True)]

# Create an empty DataFrame with the extracted column headers
df = pd.DataFrame(columns=header_text)

# Find all table rows within the <tbody> elements (contains actual rate data)
table_rows = soup.find_all('tbody')

# Initialize an empty list to store extracted row data
data = []

# Loop through each table row and extract text from the first three columns: Rate, Provider, and Payment
for row in table_rows:
    for tr in row.find_all('tr'):
        columns = tr.find_all('td')
        # Ensure there are enough columns before attempting to extract data
        if len(columns) >= 3:
            rate = columns[0].get_text(strip=True)
            provider = columns[1].get_text(strip=True)
            payment = columns[2].get_text(strip=True)
            # Append the extracted values as a new row in the data list
            data.append([rate, provider, payment])

# Create a DataFrame from the extracted data using the previously defined headers
df2f = pd.DataFrame(data, columns=header_text)
df2f.drop(columns=['Payment'], inplace=True)
df2f.rename(columns={'Rate': 'Fixed rate 2y'}, inplace=True)

result_df=result_df.merge(df2f, on='Provider', how='outer')

In [466]:
# Define the URL of the webpage to scrape mortgage rate data
url = 'https://www.ratehub.ca/best-mortgage-rates/3-year/fixed'

# Send a GET request to fetch the HTML content of the webpage
page = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page.text, 'html')

# Find the table elements that contain the data we want, specifically looking for elements with class 'table-container'
table = soup.find_all('table', class_='table-container')

# Extract and clean the text from each table element
table = [i.text.strip() for i in table]

# Locate the table header and extract text from each column name (header cell)
header = soup.find('thead')
header_text = [th.get_text(strip=True) for th in header.find_all('th') if th.get_text(strip=True)]

# Create an empty DataFrame with the extracted column headers
df = pd.DataFrame(columns=header_text)

# Find all table rows within the <tbody> elements (contains actual rate data)
table_rows = soup.find_all('tbody')

# Initialize an empty list to store extracted row data
data = []

# Loop through each table row and extract text from the first three columns: Rate, Provider, and Payment
for row in table_rows:
    for tr in row.find_all('tr'):
        columns = tr.find_all('td')
        # Ensure there are enough columns before attempting to extract data
        if len(columns) >= 3:
            rate = columns[0].get_text(strip=True)
            provider = columns[1].get_text(strip=True)
            payment = columns[2].get_text(strip=True)
            # Append the extracted values as a new row in the data list
            data.append([rate, provider, payment])

# Create a DataFrame from the extracted data using the previously defined headers
df3f = pd.DataFrame(data, columns=header_text)
df3f.drop(columns=['Payment'], inplace=True)
df3f.rename(columns={'Rate': 'Fixed rate 3y'}, inplace=True)

result_df=result_df.merge(df3f, on='Provider', how='outer')

In [467]:
# Define the URL of the webpage to scrape mortgage rate data
url = 'https://www.ratehub.ca/best-mortgage-rates/4-year/fixed'

# Send a GET request to fetch the HTML content of the webpage
page = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page.text, 'html')

# Find the table elements that contain the data we want, specifically looking for elements with class 'table-container'
table = soup.find_all('table', class_='table-container')

# Extract and clean the text from each table element
table = [i.text.strip() for i in table]

# Locate the table header and extract text from each column name (header cell)
header = soup.find('thead')
header_text = [th.get_text(strip=True) for th in header.find_all('th') if th.get_text(strip=True)]

# Create an empty DataFrame with the extracted column headers
df = pd.DataFrame(columns=header_text)

# Find all table rows within the <tbody> elements (contains actual rate data)
table_rows = soup.find_all('tbody')

# Initialize an empty list to store extracted row data
data = []

# Loop through each table row and extract text from the first three columns: Rate, Provider, and Payment
for row in table_rows:
    for tr in row.find_all('tr'):
        columns = tr.find_all('td')
        # Ensure there are enough columns before attempting to extract data
        if len(columns) >= 3:
            rate = columns[0].get_text(strip=True)
            provider = columns[1].get_text(strip=True)
            payment = columns[2].get_text(strip=True)
            # Append the extracted values as a new row in the data list
            data.append([rate, provider, payment])

# Create a DataFrame from the extracted data using the previously defined headers
df4f = pd.DataFrame(data, columns=header_text)
df4f.drop(columns=['Payment'], inplace=True)
df4f.rename(columns={'Rate': 'Fixed rate 4y'}, inplace=True)

result_df=result_df.merge(df4f, on='Provider', how='outer')

In [468]:
# Define the URL of the webpage to scrape mortgage rate data
url = 'https://www.ratehub.ca/best-mortgage-rates/5-year/fixed'

# Send a GET request to fetch the HTML content of the webpage
page = requests.get(url)

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(page.text, 'html')

# Find the table elements that contain the data we want, specifically looking for elements with class 'table-container'
table = soup.find_all('table', class_='table-container')

# Extract and clean the text from each table element
table = [i.text.strip() for i in table]

# Locate the table header and extract text from each column name (header cell)
header = soup.find('thead')
header_text = [th.get_text(strip=True) for th in header.find_all('th') if th.get_text(strip=True)]

# Create an empty DataFrame with the extracted column headers
df = pd.DataFrame(columns=header_text)

# Find all table rows within the <tbody> elements (contains actual rate data)
table_rows = soup.find_all('tbody')

# Initialize an empty list to store extracted row data
data = []

# Loop through each table row and extract text from the first three columns: Rate, Provider, and Payment
for row in table_rows:
    for tr in row.find_all('tr'):
        columns = tr.find_all('td')
        # Ensure there are enough columns before attempting to extract data
        if len(columns) >= 3:
            rate = columns[0].get_text(strip=True)
            provider = columns[1].get_text(strip=True)
            payment = columns[2].get_text(strip=True)
            # Append the extracted values as a new row in the data list
            data.append([rate, provider, payment])

# Create a DataFrame from the extracted data using the previously defined headers
df5f = pd.DataFrame(data, columns=header_text)
df5f.drop(columns=['Payment'], inplace=True)
df5f.rename(columns={'Rate': 'Fixed rate 5y'}, inplace=True)

result_df=result_df.merge(df5f, on='Provider', how='outer')

In [469]:
fixed=result_df[:20]

In [470]:
fixed=fixed.T

In [471]:
fixed.columns = fixed.iloc[0]
fixed = fixed[1:]  # Remove the first row which is now the header
fixed

Provider,Canadian Lender,CanwiseA Ratehub Company,Big 6 Bank,First National,CMLS Financial,CIBC,Desjardins,MCAP,Scotiabank,Alterna Savings,Meridian Credit Union,TD Bank,Equitable Bank,Bank of Montreal,ICICI Bank Canada,motusbank,RBC Royal Bank,National Bank of Canada,Tangerine,Simplii Financial â¢
Variable rate 3y,5.05%,NaN,NaN,NaN,NaN,5.45%,NaN,NaN,6.30%,5.30%,NaN,NaN,NaN,NaN,6.00%,NaN,NaN,NaN,NaN,NaN
Variable rate 5y,4.70%,4.95%,5.00%,5.00%,5.05%,5.15%,5.20%,5.25%,5.25%,5.30%,5.34%,5.44%,5.55%,5.86%,5.95%,6.00%,5.05%,5.45%,5.45%,5.95%
Fixed rate 1y,7.15%,NaN,5.89%,6.63%,NaN,6.29%,6.64%,7.44%,6.24%,6.44%,7.74%,6.53%,7.44%,6.08%,8.09%,7.79%,5.94%,7.24%,6.64%,6.84%
Fixed rate 2y,5.49%,5.54%,5.34%,5.92%,NaN,5.59%,5.64%,7.14%,5.59%,6.34%,7.34%,5.82%,6.44%,5.59%,7.54%,7.24%,5.54%,6.84%,5.74%,6.59%
Fixed rate 3y,4.59%,4.44%,4.39%,4.94%,NaN,4.79%,4.54%,5.09%,4.64%,4.84%,4.19%,4.57%,5.84%,4.74%,6.79%,7.04%,4.74%,6.49%,4.74%,6.94%
Fixed rate 4y,4.89%,NaN,4.49%,4.89%,NaN,4.64%,4.34%,5.04%,4.74%,5.54%,6.74%,4.69%,5.69%,4.68%,6.59%,6.79%,4.69%,4.69%,4.79%,6.54%
Fixed rate 5y,3.99%,4.29%,4.44%,4.54%,4.44%,4.34%,4.29%,4.74%,4.69%,4.69%,4.09%,4.49%,5.09%,4.45%,5.99%,6.69%,4.44%,4.69%,4.84%,4.34%


In [472]:
df_numeric = fixed.iloc[:, :].apply(lambda col: col.map(lambda x: float(x.strip('%')) if isinstance(x, str) and '%' in x else np.nan))

# Initialize lists to store the results
years = []
min_rates = []
banks = []

# Loop through each row (each year)
for year, row in df_numeric.iterrows():
    min_rate = row.min(skipna=True)
    # Get the bank that corresponds to the minimum rate
    bank = row.idxmin() if not pd.isna(min_rate) else None
    years.append(year)
    min_rates.append(f"{min_rate:.2f}%")  # Format as percentage string
    banks.append(bank)

# Create a result DataFrame
result = pd.DataFrame({
    'Year': years,
    'Min Rate': min_rates,
    'Provider': banks
})

# Filter the result DataFrame to show only rows with minimum values
result = result[result['Min Rate'] != 'nan%']

# Display the result
result

,Year,Min Rate,Provider
0,Variable rate 3y,5.05%,Canadian Lender
1,Variable rate 5y,4.70%,Canadian Lender
2,Fixed rate 1y,5.89%,Big 6 Bank
3,Fixed rate 2y,5.34%,Big 6 Bank
4,Fixed rate 3y,4.19%,Meridian Credit Union
5,Fixed rate 4y,4.34%,Desjardins
6,Fixed rate 5y,3.99%,Canadian Lender
